In [1]:
# 这里我们实现了ResNet 18，原论文中还讨论了更深的配置。尝试实现它们。（提示：参考论文中的表1）
# http://zh.gluon.ai/chapter_convolutional-neural-networks/resnet-gluon.html
# 尝试ResNet 152

from mxnet.gluon import nn
from mxnet import nd

class ResidualBottleneck(nn.Block):
    def __init__(self, channels_in, channels_out, same_shape=True, **kwargs):
        super(ResidualBottleneck, self).__init__(**kwargs)
        self.same_shape = same_shape
        strides = 1 if same_shape else 2
        self.conv1 = nn.Conv2D(channels_in, kernel_size=1,
                              strides=strides)
        self.bn1 = nn.BatchNorm()
        self.conv2 = nn.Conv2D(channels_in, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm()
        self.conv3 = nn.Conv2D(channels_out, kernel_size=1,
                              strides=1)
        self.bn3 = nn.BatchNorm()
        if not same_shape:
            self.conv4 = nn.Conv2D(channels_out, kernel_size=1,
                                  strides=strides)
            
    def forward(self, x):
#         print('same in / out shape:', self.same_shape)
#         print('x.shape:', x.shape)
        out = nd.relu(self.bn1(self.conv1(x)))
#         print(out.shape)
        out = nd.relu(self.bn2(self.conv2(out)))
#         print(out.shape)
        out = self.bn3(self.conv3(out))
#         print(out.shape)
        if not self.same_shape:
            x = self.conv4(x)
#             print('x.shape:',x.shape)
        return nd.relu(out + x)

# 构建ResNet
class ResNet152(nn.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet152, self).__init__(**kwargs)
        self.verbose = verbose
        # add name_scope on the outermost Sequential
        with self.name_scope():
            # block 1
            b1 = nn.Conv2D(64, kernel_size=7, strides=2)
            # block 2
            b2 = nn.Sequential()
            b2.add(
                nn.MaxPool2D(pool_size=3, strides=2),
                ResidualBottleneck(64, 256, same_shape=False),
                ResidualBottleneck(64, 256),
                ResidualBottleneck(64, 256)
            )
            # block 3
            b3 = nn.Sequential()
            b3.add(
                ResidualBottleneck(128, 512, same_shape=False),
            )
            for _ in range(7):
                b3.add(ResidualBottleneck(128, 512))
            # block 4
            b4 = nn.Sequential()
            b4.add(
                ResidualBottleneck(256, 1024, same_shape=False)
            )
            for _ in range(35):
                b4.add(ResidualBottleneck(256, 1024))
                    
            # block 5
            b5 = nn.Sequential()
            b5.add(
                ResidualBottleneck(512, 2048, same_shape=False),
                ResidualBottleneck(512, 2048),
                ResidualBottleneck(512, 2048)
            )
            # block 6
            b6 = nn.Sequential()
            b6.add(
                nn.AvgPool2D(pool_size=3),
                nn.Dense(num_classes)
            )
            # chain all blocks together
            self.net = nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)
    
    def forward(self, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s' % (i+1, out.shape))
        return out

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = nd.random.uniform(shape=(4, 3, 224, 224))
net = ResNet152(num_classes=10, verbose=True)
# print(net)
net.initialize()
y = net(x)
print(y.shape)
print(net)

Block 1 output: (4, 64, 109, 109)
Block 2 output: (4, 256, 27, 27)
Block 3 output: (4, 512, 14, 14)
Block 4 output: (4, 1024, 7, 7)
Block 5 output: (4, 2048, 4, 4)
Block 6 output: (4, 10)
(4, 10)
ResNet152(
  (net): Sequential(
    (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2))
    (1): Sequential(
      (0): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
      (1): ResidualBottleneck(
        (conv1): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(2, 2))
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv3): Conv2D(64 -> 256, kernel_size=(1, 1), stride=(1, 1))
        (bn3): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=256)
        

In [3]:
# 读取数据
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd
from mxnet import image
from mxnet import init

batch_size = 64
resize=224

ctx = utils.try_gpu(0)
net = ResNet152(10, verbose=False)
net.initialize(ctx=ctx, init=init.Xavier())

def transform(data, label):
#     print(data.shape)   # (28, 28, 1)
#     print(label.shape)    # (1,)
# change data from batch x height x width x channel
# to batch x channel x height x width
#     return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')
    data = image.imresize(data, resize, resize)
#     data = new_data
            
    return nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32')
#     return data.astype('float32') / 255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [4]:
# 训练
import time
from mxnet import gluon

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {
    'learning_rate': 0.005
})

for epoch in range(10):
    time_start = time.time()
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %.4f, Train acc %.4f, Test acc %.4f, Time %.0f sec" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc, time.time() - time_start))

Epoch 0. Loss: 0.9225, Train acc 0.7582, Test acc 0.8402, Time 392 sec
Epoch 1. Loss: 0.4221, Train acc 0.8466, Test acc 0.8530, Time 390 sec
Epoch 2. Loss: 0.3471, Train acc 0.8720, Test acc 0.8800, Time 390 sec
Epoch 3. Loss: 0.3024, Train acc 0.8883, Test acc 0.8827, Time 389 sec
Epoch 4. Loss: 0.2599, Train acc 0.9037, Test acc 0.8950, Time 389 sec
Epoch 5. Loss: 0.2259, Train acc 0.9160, Test acc 0.8970, Time 389 sec
Epoch 6. Loss: 0.1941, Train acc 0.9302, Test acc 0.8990, Time 389 sec
Epoch 7. Loss: 0.1637, Train acc 0.9405, Test acc 0.8986, Time 388 sec
Epoch 8. Loss: 0.1342, Train acc 0.9521, Test acc 0.9049, Time 389 sec
Epoch 9. Loss: 0.1116, Train acc 0.9603, Test acc 0.9032, Time 388 sec
